# Term Paper Project: EU Transparency Register

Research Question: Which lobby groups have been in contanct with members of the European Commission (EC) or the Directorates-Generale (DG) ?

In [1]:
# load packages
import pandas as pd
import numpy as np
import os

# Load & Transform Data
- Load Data from Transparency register
- filter for Meetings within the relevant time period

In [2]:
# load data

meetings_ec = pd.read_excel("./original_data/Meetings of Commission representatives of the Von der Leyen Commission (2019-2024).xlsx", header=1)
meetings_df = pd.read_excel("./original_data/Meetings of Directors-General of the European Commission.xlsx", header=1)

# transform subject column to lower case to improve keyword search
meetings_ec["Subject of the meeting"] = meetings_ec["Subject of the meeting"].str.lower()
meetings_df["Subject of the meeting"] = meetings_df["Subject of the meeting"].str.lower()

# filter for entries between 
min_date = "2020-06-01"
max_date = "2023-04-18"

meetings_df['Date of meeting'] = pd.to_datetime(meetings_df['Date of meeting'], format="%Y-%m-%d")
meetings_ec['Date of meeting'] = pd.to_datetime(meetings_ec['Date of meeting'], format="%Y-%m-%d")


meetings_df = meetings_df.loc[(meetings_df['Date of meeting'] >= min_date)
                     & (meetings_df['Date of meeting'] <= max_date)]

meetings_ec = meetings_ec.loc[(meetings_ec['Date of meeting'] >= min_date)
                     & (meetings_ec['Date of meeting'] <= max_date)]



In [ ]:
# select all relevant meetings by filtering for keywords in the column "Subject of the meeting"

chips_act_keywords = ["chips", "semiconduct", "chip", "2023/1781", "microelectronic", "foundry", "advanced manufacturing", "ipcei", "node", "semion", "wafer"]
keywords_to_exclude = ["schiphol", "mobility ipcei", "ipcei, hydrogen", "ipcei electrolyzer", "discussion on ipceis", "h2 ipcei"]

chips_meetings_ec = meetings_ec[
    (meetings_ec["Subject of the meeting"].str.contains('|'.join(chips_act_keywords), regex=True))  
    & (meetings_ec["Subject of the meeting"].str.contains('|'.join(keywords_to_exclude), regex=True) == False) 
    ].copy()
chips_meetings_dg = meetings_df[
    (meetings_df["Subject of the meeting"].str.contains('|'.join(chips_act_keywords), regex=True)) 
    & (meetings_df["Subject of the meeting"].str.contains('|'.join(keywords_to_exclude), regex=True) == False) 
    ].copy()

# Merge DG and EC dataframe

# rename columns, drop interest group name column (group names are retrieved from register later), create col to indicate whether meeting took place with EC or DG rep.
chips_meetings_dg.rename(columns={"Name of DG - full name": "Name of DG / EC cabinet"}, inplace=True)
chips_meetings_ec.rename(columns={"Name of cabinet": "Name of DG / EC cabinet"}, inplace=True)
chips_meetings_dg.loc[:, "EC_or_DG"] = "DG"
chips_meetings_ec.loc[:, "EC_or_DG"] = "EC"


chips_meetings = pd.concat([chips_meetings_ec, chips_meetings_dg])
chips_meetings.drop(columns=["Name of interest representative"], inplace=True)



,Name of DG / EC cabinet,Name of EC representative,Title of EC representative,Date of meeting,Location,Transparency register ID,Subject of the meeting,EC_or_DG,Name of DG - acronym
3548,Cabinet of Commissioner Thierry Breton,Joan Canton,Cabinet member,2023-04-05,Videoconference,236869539281-13,preparatory meeting for meeting between ceo of...,EC,NaN
3602,Cabinet of President Ursula von der Leyen,Anthony Whelan,Cabinet member,2023-03-30,Brussels (VTC),97921387213-35,eu chips act,EC,NaN
3768,Cabinet of Executive Vice-President Margrethe ...,"Pierre-Arnaud Proux,Michele Piergiovanni,Margr...","Cabinet member,Cabinet member,Executive Vice-P...",2023-03-21,Brussels,70843878143-13,semiconductor value chain,EC,NaN
4097,Cabinet of Commissioner Thierry Breton,"Eleonora Ocello,Lucia Caudet,Thierry Breton","Cabinet member,Cabinet member,Commissioner",2023-02-27,Barcelona,00358442856-45,"telecoms, space and semiconductors policy",EC,NaN
4098,Cabinet of Commissioner Thierry Breton,"Lucia Caudet,Eleonora Ocello,Thierry Breton","Cabinet member,Cabinet member,Commissioner",2023-02-27,Barcelona,7459401905-60,telecoms and semiconductors policy,EC,NaN
...,...,...,...,...,...,...,...,...,...
3728,"Communications Networks, Content and Technology",Roberto Viola,Director-General,2021-04-07,Brussels,64270747023-20,"digital decade, cloud, ai, semiconductors (v...",DG,CNECT
3736,"Communications Networks, Content and Technology",Roberto Viola,Director-General,2021-03-26,BRUSSELS,10751968675-85,ipcei microelectronics (virtual),DG,CNECT
3737,"Communications Networks, Content and Technology",Roberto Viola,Director-General,2021-03-26,Brussels,70843878143-13,ipcei microelectronics (virtual meeting),DG,CNECT
3741,"Communications Networks, Content and Technology",Roberto Viola,Director-General,2021-03-24,Brussels,898711618618-45,ipcei microelectronics (virtual meeting),DG,CNECT


In [4]:
# load transparency register

transparency_register = pd.DataFrame(columns=["Transparency register ID", "Head Office"])

# Head office country, Identification number:


for entry in os.scandir("./original_data"):  
    if entry.is_file() and entry.name.endswith(".xls"):  # check if it's a xlsx file
        print(entry.path)
        xls = pd.read_excel(entry.path)

        if "Identification number:" in xls.columns:
            xls = xls[["Head office country", "Identification number:", "(Organisation) name"]] 
            xls.rename(columns={"Identification number:": "Transparency register ID",
                                 "Head office country": "Head Office"}, inplace=True)
        elif "Identification number" in xls.columns:
            xls = xls[["Head office country", "Identification number", "(Organisation) name"]]
            xls.rename(columns={"Identification number": "Transparency register ID",
                                 "Head office country": "Head Office"}, inplace=True)
        elif "Identification code" in xls.columns:
            xls = xls[["Head office country", "Identification code", "(Organisation) name"]]
            xls.rename(columns={"Identification code": "Transparency register ID",
                                 "Head office country": "Head Office"}, inplace=True)
        else:
            raise KeyError("ERROR MISSING TRANSPARENCY REGISTER ID COLUMN!")
        transparency_register = pd.concat([transparency_register, xls]).drop_duplicates(subset="Transparency register ID")

transparency_register["Head Office"] = transparency_register["Head Office"].str.upper()
transparency_register.rename(columns={"(Organisation) name": "Interest Group Name"})
# transparency_register.to_excel("./original_data/merged_register.xlsx", index=False)



./original_data\Organisations in Transparency Register - 2019 - JANUARY.xls
./original_data\Organisations in Transparency Register - 2019 - JUNE.xls
./original_data\Organisations in Transparency Register - 2020 - JANUARY.xls
./original_data\Organisations in Transparency Register - 2020 - JUNE.xls
./original_data\Organisations in Transparency Register - 2021 - JANUARY.xls
./original_data\Organisations in Transparency Register - 2021 - JUNE.xls
./original_data\Organisations in Transparency Register - 2022 - JANUARY.xls
./original_data\Organisations in Transparency Register - 2022 - JUNE.xls
./original_data\Organisations in Transparency Register - 2023 - JANUARY.xls
./original_data\Organisations in Transparency Register - 2023 - JUNE.xls
./original_data\Organisations in Transparency Register - 2024 - JAN.xls


,Transparency register ID,Head Office,Interest Group Name
0,117160930653-96,BELGIUM,Hydrogen Council
1,596942331763-90,SWEDEN,Fiskekommunerna (FK)
2,588872531784-23,CZECH REPUBLIC,Ocelářská unie
3,993615331973-07,GERMANY,Deutscher Aero Club e. V. (DAeC)
4,088905531785-87,NETHERLANDS,European Arenas Association (EAA)
...,...,...,...
12437,018493851897-86,BELGIUM,International Freedom of Speech Alliance
12450,069810551569-10,ITALY,Associazione Apnoici Italiani APS
12455,310344150876-09,UNITED KINGDOM,Plan International EU office
12458,638934751718-03,POLAND,CASE - Centrum Analiz Społeczno-Ekonomicznych


In [5]:
# Handle Meetings with multiple entries in Register ID Column: Create as many rows as there are interests groups participating

# iterate through rows with multiple REG in the same cell
new_rows = pd.DataFrame(columns=chips_meetings.columns)

for index, row in chips_meetings[chips_meetings["Transparency register ID"].str.contains(",")].iterrows():
    reg_ids = str(row["Transparency register ID"]).split(",")
    reg_id_rows = pd.DataFrame(columns=new_rows.columns)
    
    for id in reg_ids:
        row["Transparency register ID"] = id
        reg_id_rows = pd.concat([reg_id_rows, row.to_frame().transpose()])
    
    new_rows = pd.concat([new_rows, reg_id_rows])

chips_meetings_appended = pd.concat([chips_meetings[chips_meetings["Transparency register ID"].str.contains(",") == False], new_rows])

meetings_df = pd.merge(left=chips_meetings_appended, right=transparency_register, how="left", on="Transparency register ID")

# meetings_df = meetings_df[:, ]
